# Optimization

of linear-quadratic problems, using the [OSQP.jl](https://github.com/osqp/OSQP.jl) package.

The example is (for pedagogical reasons) the same as in the other notebooks on optimization. Otherwise, the methods illustrated here are well suited for cases when the objective involves the portfolio variance ($ w'\Sigma w $) or when the estimation problem is based on minimizing the sum of squared residuals ($u'u$).

## Load Packages and Utility Functions

In [1]:
using Printf, LinearAlgebra, SparseArrays, OSQP

include("jlFiles/printmat.jl")

printyellow (generic function with 1 method)

# The OSQP.jl Optimization Package

The [OSQP.jl](https://github.com/osqp/OSQP.jl) package is tailor made for solving linear-quadratic problems (and more). It solves problems of the type

$\min 0.5\theta' P \theta + q' \theta$ subject to $l \leq A \theta \leq u$.

The next cell defines a convenience function to facilitate calling on the optimization algorithm. It converts $(P,A)$ to `Sparse` and $(q,l,u)$ to `Float64` (as required by the package).

In [2]:
"""
  DoOSQP(P,q,A,l,u,settings)

Utility function to help setting up the minimisation problem according
to the OSQP API. Basically, it transforms to Float64 and sparse (for P and A)
and extracts the solution.

It solves `min 0.5θ'P*θ + q'θ` subject to `l <= A*θ <= u`.


# Input
- `P::Matrix`:      nxn
- `q::Vector`:      n-vector
- `A::Matrix`:      Kxn
- `l::Vector`:      K-vector
- `u::Vector`:      K-vector
- `settings::Dict`: eg. Dict(:verbose => true)

"""
function DoOSQP(P,q,A,l,u,settings)

  P2 = sparse(P)              #convert to P,A to sparse
  q2 = Float64.(q)            #convert to q,l,u to Float64
  A2 = sparse(A)
  l2 = Float64.(l)
  u2 = Float64.(u)

  model = OSQP.Model()
  OSQP.setup!(model; P=P2, q=q2, A=A2, l=l2, u=u2, settings...)
  result = OSQP.solve!(model)

  x = result.info.status == :Solved ? result.x : NaN

  return x, result

end

DoOSQP

# A Linear-Quadratic Minimization Problem

without constraints.

We specify a matrix $P$ and a vector $q$ and write the loss function as $0.5 \theta' P \theta + q' \theta$ where $\theta$ are the choice variables.

We consider several cases below: no restrictions on $\theta$, bounds on $\theta$, a linear equality restriction and a non-linear inequality restriction.

In [3]:
P = 2*[1  0;           #we want to minimize θ'[]*θ + q'θ, hence the 2*[]
       0  16]          #this is the same as minimizing (x-2)^2 + (4y+3)^2
q =   [-4, 24]

2-element Vector{Int64}:
 -4
 24

## Unconstrained Minimization

In [4]:
n = length(q)
A = zeros(1,n)                  #effectively no restriction
l = [0]                         #vectors
u = [0]

settings = Dict(:verbose => true)
(b_sol,result) = DoOSQP(P,q,A,l,u,settings)   #notice 2*Q

println("Unconstrained minimization: the solution should be (2,-3/4)")
printmat(b_sol)

Unconstrained minimization: the solution should be (2,-3/4)
     2.000
    -0.750

-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 2, constraints m = 1
          nnz(P) + nnz(A) = 2
settings: linear system solver = qdldl,
          eps_abs = 1.0e-003, eps_rel = 1.0e-003,
          eps_prim_inf = 1.0e-004, eps_dual_inf = 1.0e-004,
          rho = 1.00e-001 (adaptive),
          sigma = 1.00e-006, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter  objective    pri res    dua res    rho        time
   1 -8.3200e+000  0.00e+000  1.44e+001  1.00e-001  3.26e-005s
  25 -1.3000e+001  0.00e+0

## Constrained Minimization

In [5]:
A = [1 0;
      0 1]
l = [2.75,-Inf]    #2.75 <= θ₁ <= Inf, -Inf <= θ₂ <= -0.3
u = [Inf,-0.3]

settings = Dict(:verbose => false)
(b_sol,result) = DoOSQP(P,q,A,l,u,settings)
println("with bounds on the solution: the solution should be (2.75,-0.75)")
printmat(b_sol)

with bounds on the solution: the solution should be (2.75,-0.75)
     2.750
    -0.750



In [6]:
A = [1 2]                    #equality constraint
l = [3]
u = [3]

(b_sol,result) = DoOSQP(P,q,A,l,u,settings)
println("equality constraint: the solution should be (4,-1/2)")
printmat(b_sol)

equality constraint: the solution should be (4,-1/2)
     4.000
    -0.500



# Updating the Problem (extra)

When you just want to change some of the inputs to the optimization problem, then it may pay off to do `update!()`. This is especially useful when you resolve the problem many times (in a loop, say).

In [7]:
model = OSQP.Model()
OSQP.setup!(model; P=sparse(P), q=Float64.(q), A=sparse(A), l=Float64.(l), u=Float64.(u), settings...)
result = OSQP.solve!(model)
println(result.x)

q_new = [-2.0,20.0]                          #resolve the problem with different q values
OSQP.update!(model;q=q_new)
result = OSQP.solve!(model)
println(result.x)

[4.0000000038916035, -0.4999999995053219]
[3.5999987176527286, -0.299999449652652]


# An Alternative Package

The [Clarabel](https://github.com/oxfordcontrol/Clarabel.jl) package is an interesting alternative. Is has a slightly different syntax, but the the `DoClarabel()` function in `jlFiles/DoClarabel.jl` provides a way to call it using the same syntax as for OSQP.jl.